[competition](https://www.kaggle.com/competitions/customer-churn-prediction-2020) | [solution](https://www.kaggle.com/code/volodymyrholomb/autoeda-pipeline)

In [1]:
# !pip install -U scikit-learn

In [2]:
import warnings
import pandas as pd
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import (
    PowerTransformer,
    KBinsDiscretizer,
    TargetEncoder)
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE
# from ydata_profiling import ProfileReport

In [3]:
train = pd.read_csv('../competition/demo_train.csv')
valid = pd.read_csv('../competition/demo_valid.csv')

# train = pd.read_csv('/kaggle/input/customer-churn-prediction-2020/train.csv')
# valid = pd.read_csv('/kaggle/input/customer-churn-prediction-2020/test.csv')

In [4]:
# report = ProfileReport(train)
# report.to_notebook_iframe()

In [5]:
target = train.pop('churn')

In [6]:
model = make_pipeline(
    make_column_transformer(
        (TargetEncoder(random_state=42),
         make_column_selector(dtype_include=object)),
        remainder='passthrough',
    n_jobs=-1),
    SelectKBest(),
    PowerTransformer(),
    SMOTE(random_state=42),
    KBinsDiscretizer(
        encode='onehot-dense',
        strategy='uniform',
        subsample=None,
        random_state=42),
    GradientBoostingClassifier(
        random_state=42))

In [7]:
# model.get_params().keys()

params = {
    'selectkbest__k': [10, 15],
    'smote__k_neighbors': [7, 9],
    'gradientboostingclassifier__subsample': [0.65, 0.85],
    'gradientboostingclassifier__max_depth': [5, 7]
}

rs = RandomizedSearchCV(
    model,
    params,
    n_jobs=-1,
    refit=False,
    random_state=42,
    verbose=1)

search = rs.fit(train, target)
search.best_params_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


{'smote__k_neighbors': 7,
 'selectkbest__k': 15,
 'gradientboostingclassifier__subsample': 0.85,
 'gradientboostingclassifier__max_depth': 7}

In [8]:
model.set_params(**search.best_params_)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(n_jobs=-1, remainder='passthrough',
                                   transformers=[('targetencoder',
                                                  TargetEncoder(random_state=42),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000250B4178B10>)])),
                ('selectkbest', SelectKBest(k=15)),
                ('powertransformer', PowerTransformer()),
                ('smote', SMOTE(k_neighbors=7, random_state=42)),
                ('kbinsdiscretizer',
                 KBinsDiscretizer(encode='onehot-dense', random_state=42,
                                  strategy='uniform', subsample=None)),
                ('gradientboostingclassifier',
                 GradientBoostingClassifier(max_depth=7, random_state=42,
                                            subsample=0.85))])

In [9]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')

    cv_results = cross_val_score(
        estimator=model,
        X=train,
        y=target,
        scoring='balanced_accuracy',
        cv=10,
        n_jobs=-1)

cv_results.mean()

0.844043287645858

In [10]:
model.fit(train, target)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(n_jobs=-1, remainder='passthrough',
                                   transformers=[('targetencoder',
                                                  TargetEncoder(random_state=42),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000250B4178B10>)])),
                ('selectkbest', SelectKBest(k=15)),
                ('powertransformer', PowerTransformer()),
                ('smote', SMOTE(k_neighbors=7, random_state=42)),
                ('kbinsdiscretizer',
                 KBinsDiscretizer(encode='onehot-dense', random_state=42,
                                  strategy='uniform', subsample=None)),
                ('gradientboostingclassifier',
                 GradientBoostingClassifier(max_depth=7, random_state=42,
                                            subsample=0.85))])

In [11]:
output = pd.DataFrame({'id': valid['id'],
                       'churn': model.predict(valid)})
# output.to_csv('submission.csv', index=False)